# script to prepare the files for insulation QTL analysis

genotypes are only available through protected access

QTL was generated using QTLtools

In [2]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

os.chdir("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis")
base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 28 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## prepare the normalized scores for insQTL

In [4]:
metadata_hic = pd.read_csv(f"{base_dir}/metadata/cleaned_HiC_metadata.csv", index_col = 0)
ins_scores = pd.read_csv(f"{base_dir}/HiC_analysis/insulation_score/aggregated_norm_ins_scores.csv.gz", index_col = 0)

/tmp/ipykernel_330/764316690.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ins_scores = pd.read_csv(f"{base_dir}/HiC_analysis/insulation_score/aggregated_norm_ins_scores.csv.gz", index_col = 0)


In [9]:
for_qtl = ins_scores[["chrom", "start", "end"] + metadata_hic[metadata_hic["cell_type"] == "CD8"]["folder_name"].to_list()].copy()
for_qtl.columns = ["#Chr", "start", "end"] + metadata_hic[metadata_hic["cell_type"] == "CD8"]["folder_name"].to_list()
for_qtl["strand"] = "+"
for_qtl["pid"] = for_qtl.index
for_qtl["gid"] = for_qtl.index
for_qtl = for_qtl[["#Chr", "start", "end", "pid", "gid", "strand"] + metadata_hic[metadata_hic["cell_type"] == "CD8"]["folder_name"].to_list()]
for_qtl = for_qtl.rename(columns=dict(zip(metadata_hic["folder_name"], metadata_hic.patient)))
for_qtl = for_qtl.dropna()
for_qtl.to_csv(f"{base_dir}/QTL_analysis/HiC/insulation_score_CD8.bed", sep = "\t", index = False, na_rep='NA')

for_qtl = ins_scores[["chrom", "start", "end"] + metadata_hic[metadata_hic["cell_type"] == "CD4"]["folder_name"].to_list()].copy()
for_qtl.columns = ["#Chr", "start", "end"] + metadata_hic[metadata_hic["cell_type"] == "CD4"]["folder_name"].to_list()
for_qtl["strand"] = "+"
for_qtl["pid"] = for_qtl.index
for_qtl["gid"] = for_qtl.index
for_qtl = for_qtl[["#Chr", "start", "end", "pid", "gid", "strand"] + metadata_hic[metadata_hic["cell_type"] == "CD4"]["folder_name"].to_list()]
for_qtl = for_qtl.rename(columns=dict(zip(metadata_hic["folder_name"], metadata_hic.patient)))
for_qtl = for_qtl.dropna()

for_qtl.to_csv(f"{base_dir}/QTL_analysis/HiC/insulation_score_CD4.bed", sep = "\t", index = False, na_rep='NA')

In [10]:
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/insulation_score_CD8.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/insulation_score_CD8.bed.gz
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/insulation_score_CD4.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/insulation_score_CD4.bed.gz

## running a permutation test to identify the best number of PCA component as covariates identified the following:
for CD8: 0 genotype covariates and 5 phenotype covariates
for CD4: 0 genotype covariates and 5 phenotype covariates

In [ ]:
# see bash scripts for running of QTL tools

In [1]:
# merge the final results
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_nominal_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_nominal_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_nominal_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_nominal_CD8_merged.txt

!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_permuted_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_permuted_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_permuted_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_permuted_CD8_merged.txt

In [7]:
ins_permuted_CD8 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_permuted_CD8_merged.txt", sep = " ")
len(ins_permuted_CD8[ins_permuted_CD8["adj_beta_pval"] < 0.1])

26021

In [8]:
ins_permuted_CD4 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/ins_permuted_CD4_merged.txt", sep = " ")
len(ins_permuted_CD4[ins_permuted_CD4["adj_beta_pval"] < 0.1])

24807